In [1]:
import pandas as pd

In [2]:
bank_df = pd.read_csv(r'C:\Users\DEEPAK KUMAR\Downloads\bank.csv')

In [3]:
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,1,0,unknown,no
3,47,bluecollar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,1,0,unknown,no
4,33,unknown,single,single,no,1,no,no,unknown,5,may,198,1,1,0,unknown,no


In [4]:
bank_df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,bluecollar,married,secondary,no,668,no,no,telephone,17,nov,508,4,1,0,unknown,no
45210,37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other,no


In [5]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
bank_df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [7]:
bank_df.rename(columns={'job':'Job_Type','housing':'Housing_Loan','loan':'Personal_Loan'},inplace = True)
bank_df.rename(columns={'duration':'Call_Duration','campaign':'Current_Followups','pdays':'Contact_Day_Diff'}, inplace= True)
bank_df.rename(columns={'previous':'Previous_Followups','poutcome':'Prev_Camp_Status','y':'Current_Camp_Status'}, inplace= True)

Create New Age_Band to improve prediction insight

In [8]:
bank_df['Age_Band'] = 0
bank_df.loc[(bank_df.age>=18) & (bank_df.age<=30),'Age_Band'] = 'Young Adults'
bank_df.loc[(bank_df.age>30) & (bank_df.age<=45),'Age_Band'] = 'Mid Age Adults'
bank_df.loc[(bank_df.age>45) & (bank_df.age<=60),'Age_Band'] = 'Veterans'
bank_df.loc[bank_df.age>60,'Age_Band'] = 'Senior Citizen'

In [9]:
bank_df.Age_Band.value_counts()

Mid Age Adults    23733
Veterans          13260
Young Adults       7030
Senior Citizen     1188
Name: Age_Band, dtype: int64

# Checking grouping and treating the missing values from the balance column

since all the entries in the balance are numeric thus we will first define a function for the grouping to the balance column

In [12]:
bank_df['Balance_Band'] = 0
bank_df.loc[bank_df.balance<=0,'Balance_Band'] = 'Negative'
bank_df.loc[(bank_df.balance>0) & (bank_df.balance<=500),'Balance_Band'] = 'Low Balance'
bank_df.loc[bank_df.balance>500,'Balance_Band'] = 'High Balance'

In [13]:
bank_df.Balance_Band.value_counts()

High Balance    22239
Low Balance     19458
Negative         3514
Name: Balance_Band, dtype: int64

In [14]:
bank_df.Job_Type.value_counts()

bluecollar       9732
management       9458
technician       7597
admin            5171
services         4154
retired          2264
self employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: Job_Type, dtype: int64

In the above we can see there are 288 unknown values available in the job category and it may affect the outcome, so we need to treat then with mode of the column, since the records are categorical in nature

In [16]:
#finding the mode so we can replace it with unknown entries.

bank_df.Job_Type.mode()

0    bluecollar
Name: Job_Type, dtype: object

In [17]:
#Replacing null value with mode column by creating the function

def unknown2bluecollar(x):
    if x == 'unknown':
        return 'bluecollar'
    else:
        return x

In [18]:
#Applying function on jobtypes

bank_df['Job_Type'] = bank_df['Job_Type'].apply(unknown2bluecollar)

In [19]:
#Checking after replacing the unknown with mode

bank_df['Job_Type'].value_counts()

bluecollar       10020
management        9458
technician        7597
admin             5171
services          4154
retired           2264
self employed     1579
entrepreneur      1487
unemployed        1303
housemaid         1240
student            938
Name: Job_Type, dtype: int64

In [20]:
bank_df['Job_Type'].unique()

array(['management', 'technician', 'entrepreneur', 'bluecollar',
       'retired', 'admin', 'services', 'self employed', 'unemployed',
       'housemaid', 'student'], dtype=object)

In [21]:
#Creating a function
def job_group(x):
    if x == 'admin' or x == 'management' or x == 'services':
        return 'White Collar'
    elif x == 'bluecollar' or x == 'housemaid' or x == 'technician':
        return 'Blue Collar'
    elif x == 'entrepreneur' or x == 'self employed':
        return 'entrepreneur'
    else:
        return x

In [22]:
# Applying the grouping function

bank_df['Job_Type'] = bank_df['Job_Type'].apply(job_group)

In [23]:
#Checking the result after applying function
bank_df.Job_Type.value_counts()

Blue Collar     18857
White Collar    18783
entrepreneur     3066
retired          2264
unemployed       1303
student           938
Name: Job_Type, dtype: int64

# Checking and treating the missing values from education column

In [24]:
#Checking null value from columns
bank_df['education'].value_counts()

secondary    23202
tertiary     13301
primary       6851
unknown       1856
single           1
Name: education, dtype: int64

in the above result shows there are 928 unknown entries, to treat them we need to replace them with mode of the column

In [25]:
#finding the mode of education
bank_df.education.mode()

0    secondary
Name: education, dtype: object

In [26]:
#Creating the function
def rep_edu(x):
    if x == 'unknown':
        return 'secondary'
    else:
        return x

In [27]:
bank_df.education = bank_df.education.apply(rep_edu)

In [28]:
bank_df.education.value_counts()

secondary    25058
tertiary     13301
primary       6851
single           1
Name: education, dtype: int64

# Converting the value in the last call duration column from second to minutes & rounding to 0

In [29]:
bank_df.Call_Duration = (bank_df.Call_Duration/60).round(0)

In [31]:
bank_df.head()

,age,Job_Type,marital,education,default,balance,Housing_Loan,Personal_Loan,contact,day,month,Call_Duration,Current_Followups,Contact_Day_Diff,Previous_Followups,Prev_Camp_Status,Current_Camp_Status,Age_Band,Balance_Band
0,58,White Collar,married,tertiary,no,2143,yes,no,unknown,5,may,4.0,1,1,0,unknown,no,Veterans,High Balance
1,44,Blue Collar,single,secondary,no,29,yes,no,unknown,5,may,3.0,1,1,0,unknown,no,Mid Age Adults,Low Balance
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,1.0,1,1,0,unknown,no,Mid Age Adults,Low Balance
3,47,Blue Collar,married,secondary,no,1506,yes,no,unknown,5,may,2.0,1,1,0,unknown,no,Veterans,High Balance
4,33,Blue Collar,single,single,no,1,no,no,unknown,5,may,3.0,1,1,0,unknown,no,Mid Age Adults,Low Balance


Grouping the value in the call duration

In [32]:
#creating a function 

def group_cd(d):
    if d>=0 and d<=2:
        return  'Short Call Time'
    elif d>2 and d<=5:
        return 'Medium Call Time'
    else:
        return 'High Call Time'

In [34]:
#Applying the grouping value into call_dur column

bank_df.Call_Duration = bank_df.Call_Duration.apply(group_cd)

In [35]:
bank_df.Call_Duration.value_counts()

Short Call Time     18610
Medium Call Time    15862
High Call Time      10739
Name: Call_Duration, dtype: int64

# Dropping the Contact column since it is not helping to analyze anything

In [36]:
bank_df.drop(['contact'],axis = 1, inplace= True)

In [37]:
bank_df.head(3)

,age,Job_Type,marital,education,default,balance,Housing_Loan,Personal_Loan,day,month,Call_Duration,Current_Followups,Contact_Day_Diff,Previous_Followups,Prev_Camp_Status,Current_Camp_Status,Age_Band,Balance_Band
0,58,White Collar,married,tertiary,no,2143,yes,no,5,may,Medium Call Time,1,1,0,unknown,no,Veterans,High Balance
1,44,Blue Collar,single,secondary,no,29,yes,no,5,may,Medium Call Time,1,1,0,unknown,no,Mid Age Adults,Low Balance
2,33,entrepreneur,married,secondary,no,2,yes,yes,5,may,Short Call Time,1,1,0,unknown,no,Mid Age Adults,Low Balance


In [38]:
# Grouping the current Followups
def group_followups(x):
    if x>0 and x<5:
        return 'Upto 5 Followups'
    else:
        return 'more than 5 Followups'

In [39]:
#Applying grouping function
bank_df.Current_Followups = bank_df.Current_Followups.apply(group_followups)

In [40]:
bank_df.Current_Followups.value_counts()

Upto 5 Followups         39092
more than 5 Followups     6119
Name: Current_Followups, dtype: int64

# Grouping the Contact_Day_Diff

In [41]:
#Creating function for contact_day_diff column group

def group_cdf(x):
    if x == -1:
        return 'Not Connected'
    elif x>=0 and x<=90:
        return '0-3 Month Back'
    elif x>90 and x<=180:
        return '3-6 Month Back'
    else:
        return 'More than 6 Month'

In [42]:
#Applying the function to the column

bank_df.Contact_Day_Diff = bank_df.Contact_Day_Diff.apply(group_cdf)

In [43]:
bank_df.Contact_Day_Diff.value_counts()

0-3 Month Back       37672
More than 6 Month     5059
3-6 Month Back        2480
Name: Contact_Day_Diff, dtype: int64

# Grouping the Previous followups Column

In [44]:
#Creating the previous_followups column

def pre_foll(x):
    if x>0 and x<=5:
        return 'Upto 5 followups'
    else:
        return 'More than 5 followups'

In [46]:
#Applying the function in column

bank_df.Previous_Followups = bank_df.Previous_Followups.apply(pre_foll)

# Operating the pre_camp_status column

In [48]:
bank_df.Prev_Camp_Status.value_counts()

unknown    36959
failure     4901
other       1840
success     1511
Name: Prev_Camp_Status, dtype: int64

int the above result shows we have 36k unknown records, i will consider them as not contacted and 1.8k other does not defines anything thus i will consider it as a failure

In [49]:
#Replacing the unwanted records: unknown = not connected , other = failure

bank_df.Prev_Camp_Status = bank_df.Prev_Camp_Status.replace('unknown','not connected').replace('other','failure')

In [50]:
bank_df.Prev_Camp_Status.value_counts()

not connected    36959
failure           6741
success           1511
Name: Prev_Camp_Status, dtype: int64

In [52]:
#Suppose replacing the categories once again to assign meaningful name

bank_df.Prev_Camp_Status = bank_df.Prev_Camp_Status.replace('success','P Subscribed').replace('failure','P N Subscribed')

In [54]:
bank_df.Prev_Camp_Status.value_counts()

not connected     36959
P N Subscribed     6741
P Subscribed       1511
Name: Prev_Camp_Status, dtype: int64

# Operating the current_camp_status

Replacing 'yes' = Subscribed and 'no' = Not Subscribed

In [55]:
bank_df.Current_Camp_Status = bank_df.Current_Camp_Status.replace('yes','Subscribed').replace('no','Not Subscribed')

In [56]:
bank_df.Current_Camp_Status.value_counts()

Not Subscribed    39922
Subscribed         5289
Name: Current_Camp_Status, dtype: int64

In [57]:
bank_df.head()

,age,Job_Type,marital,education,default,balance,Housing_Loan,Personal_Loan,day,month,Call_Duration,Current_Followups,Contact_Day_Diff,Previous_Followups,Prev_Camp_Status,Current_Camp_Status,Age_Band,Balance_Band
0,58,White Collar,married,tertiary,no,2143,yes,no,5,may,Medium Call Time,Upto 5 Followups,0-3 Month Back,More than 5 followups,not connected,Not Subscribed,Veterans,High Balance
1,44,Blue Collar,single,secondary,no,29,yes,no,5,may,Medium Call Time,Upto 5 Followups,0-3 Month Back,More than 5 followups,not connected,Not Subscribed,Mid Age Adults,Low Balance
2,33,entrepreneur,married,secondary,no,2,yes,yes,5,may,Short Call Time,Upto 5 Followups,0-3 Month Back,More than 5 followups,not connected,Not Subscribed,Mid Age Adults,Low Balance
3,47,Blue Collar,married,secondary,no,1506,yes,no,5,may,Short Call Time,Upto 5 Followups,0-3 Month Back,More than 5 followups,not connected,Not Subscribed,Veterans,High Balance
4,33,Blue Collar,single,single,no,1,no,no,5,may,Medium Call Time,Upto 5 Followups,0-3 Month Back,More than 5 followups,not connected,Not Subscribed,Mid Age Adults,Low Balance


# Save the file

Now i am going to save the file. so that, i can perform data visualization after hooking upto Power BI

In [59]:
bank_df.to_csv('Banking_Marketing_Data_File.csv')